In [ ]:
import os
os.chdir('/data/kkovacs/Python')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import scanpy as sc
import copy
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import gdown
import sankey
import anndata
import pandas as pd
import pandas as pd
from pySankey.sankey import sankey
from matplotlib.pyplot import rc_context
import seaborn as sns
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(5, 5))
torch.set_printoptions(precision=10, sci_mode=False, edgeitems=12)
pd.options.mode.chained_assignment = None

In [3]:
directory = '/data/kkovacs/Python'

#.../MouseAtlas/

Initialize all studies

In [30]:
covid = sc.read(directory + "/Data/Mouse_Covid.h5ad")
copd_covid = sc.read(directory + "/Data/Mouse_Copd_Covid.h5ad")
copd = sc.read(directory + "/Data/Mouse_Copd.h5ad")
age = sc.read(directory + "/Data/Mouse_Age.h5ad")
cssc = sc.read(directory + "/Data/Mouse_COPD_SHAM_SARSCov2.h5ad")
herpes = sc.read(directory + "/Data/Mouse_Herpesvirus.h5ad")
hyper = sc.read(directory + "/Data/Mouse_Hyperoxia.h5ad")
tb = sc.read(directory + "/Data/Mouse_TB.h5ad")#c
cancer = sc.read(directory + "/Data/Mouse_Cancer.h5ad")
hypo = sc.read(directory + "/Data/Mouse_Hypoxia.h5ad")
rad = sc.read(directory + "/Data/Mouse_Radiation.h5ad")
sendai = sc.read(directory + "/Data/Mouse_Post_Sendai.h5ad")
tumor = sc.read(directory + "/Data/Mouse_TumorKO.h5ad")
fibro = sc.read(directory + "/Data/Mouse_fibrosis.h5ad")
asthma = sc.read(directory + "/Data/Mouse_asthma.h5ad")
fibage = sc.read(directory + "/Data/Mouse_fibro_age.h5ad")
influ = sc.read(directory + "/Data/Mouse_influenza.h5ad")

print(covid)

AnnData object with n_obs × n_vars = 36692 × 33260
    obs: 'nCount_covid', 'nFeature_covid', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'CellStatus', 'pANN_0.25_0.09_241', 'DF.classifications_0.25_0.09_241', 'DoubletStatus', 'pANN_0.25_0.09_520', 'DF.classifications_0.25_0.09_520', 'pANN_0.25_0.09_148', 'DF.classifications_0.25_0.09_148', 'pANN_0.25_0.09_368', 'DF.classifications_0.25_0.09_368', 'pANN_0.25_0.09_1110', 'DF.classifications_0.25_0.09_1110', 'pANN_0.25_0.09_33', 'DF.classifications_0.25_0.09_33', 'pANN_0.25_0.09_1015', 'DF.classifications_0.25_0.09_1015', 'nGene', 'nUMI', 'Group', 'res.0', 'res.0.1', 'res.0.2', 'res.0.3', 'res.0.4', 'res.0.5', 'res.0.6', 'res.0.7', 'res.0.8', 'res.0.9', 'res.1', 'seurat_clusters', 'infection', 'Sample.Gender', 'percent.hb', 'percent.rp', 'cluster_labels_res.0.4', 'predicted.classification_Angelidis.score', 'predicted.classification_Angelidis', 'mouseRNA.labels', 'immgen.labels', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_sn

In [31]:
covid.X.data = covid.raw.X.data
copd.X.data = copd.raw.X.data
copd_covid.X.data = copd_covid.raw.X.data
age.X.data = age.raw.X.data
cssc.X.data = cssc.raw.X.data
herpes.X.data = herpes.raw.X.data
hyper.X.data = hyper.raw.X.data
tb.X.data = tb.raw.X.data
hypo.X.data = hypo.raw.X.data
cancer.X.data = cancer.raw.X.data
rad.X.data = rad.raw.X.data
sendai.X.data = sendai.raw.X.data
tumor.X.data = tumor.raw.X.data
fibro.X.data = fibro.raw.X.data
asthma.X.data = asthma.raw.X.data
fibage.X.data = fibage.raw.X.data
influ.X.data = influ.raw.X.data

Ensure all metadata is harmonised with all sets before proceeding

In [32]:
copd.obs['Group'] = copd.obs['Timepoint']
age.obs['Group'] = age.obs['grouping']
herpes.obs['Group'] = herpes.obs['treatment']
hyper.obs['Group'] = hyper.obs['treatment']
tb.obs['Group'] = tb.obs['treatment']
cancer.obs['Group'] = cancer.obs['Treatment']
hypo.obs['Group'] = hypo.obs['treatment']
rad.obs['Group'] = rad.obs['treatment']
sendai.obs['Group'] = sendai.obs['Treatment']
tumor.obs['Group'] = tumor.obs['treatment']
fibro.obs['Group'] = fibro.obs['treatment']
asthma.obs['Group'] = asthma.obs['SampleIDs']
influ.obs['Group'] = influ.obs['treatment']

#invert these signs to produce the disease only sets (== -> !=, | -> &)
covid = covid[(covid.obs['Group'] == 'FemaleControl')|\
	(covid.obs['Group'] == 'MaleControl')]
copd_covid = copd_covid[(copd_covid.obs['Group'] == 'AirControl')]
copd = copd[(copd.obs['Group'] == '12 Week Air')]
age = age[(age.obs['Group'] == '3m')]
cssc = cssc[(cssc.obs['Group'] == 'AirControl')]
herpes = herpes[(herpes.obs['Group'] == 'Untreated')]
hyper = hyper[(hyper.obs['Group'] == 'Room Air')]
tb = tb[(tb.obs['Group'] == 'Uninfected')]
cancer = cancer[(cancer.obs['Group'] == 'Control')]
hypo = hypo[(hypo.obs['Group'] == 'Room Air')]
rad = rad[(rad.obs['Group'] == 'Control')]
sendai = sendai[(sendai.obs['Group'] == 'PBS')]
tumor = tumor[(tumor.obs['Group'] == 'Control')]
fibro = fibro[(fibro.obs['Group'] == 'Control')]
asthma = asthma[(asthma.obs['Group'] == 'Sal/SPG')]
fibage = fibage[(fibage.obs['Group'] == 'Control')]
influ = influ[(influ.obs['Group'] == 'Control')]

Split each set using their 'Group' metadata into Control and 'Disease' samples

Save control and disease as seperate anndata objects. To be used for the all gene objects.

Once saved find the top 2000 highly variable genes that occur in all datasets, using the top 6000 highly variable from each set, and cross correlating with each set to find the 2000

In [33]:
###These objects are to represent both Healthy and Disease to be completed each indivi
sets = [covid,copd_covid,copd,age,cssc,herpes,hyper,tb,cancer,hypo,rad,\
	sendai,tumor,fibro,asthma,fibage,influ]
vnames = []


def check_non_integer_values(adatas):
    # Loop through each adata object in the list
    for idx, adata in enumerate(adatas):
        # Assuming adata is an AnnData object
        if not isinstance(adata, anndata.AnnData):
            raise ValueError(f"Input at index {idx} is not an AnnData object.")
        study = str(adata.obs['study'].unique()[0])
        non_integer_values = np.any(adata.X.data % 1 != 0)
        # Print information about non-integer values
        if non_integer_values:
            print(f"Study: {study} - Non-integer values found.")
            # Add additional information or actions as needed
        else:
            print(f"Study: {study} - All values are integers.")
            adata.layers['raw'] = adata.X

sets = [covid,copd_covid,copd,age,cssc,herpes,hyper,tb,cancer,hypo,rad,\
	sendai,tumor,fibro,asthma,fibage,influ]

check_non_integer_values(sets)

Study: Mouse_Covid - All values are integers.
Study: Mouse_Copd_Covid - All values are integers.
Study: Mouse_Copd - All values are integers.
Study: Mouse_Age - All values are integers.
Study: Mouse_COPD_SHAM_SARSCov2 - All values are integers.
Study: Mouse_Herpesvirus - All values are integers.
Study: Mouse_Hyperoxia - All values are integers.
Study: Mouse_TB - All values are integers.
Study: Mouse_Cancer - All values are integers.
Study: Mouse_Hypoxia - All values are integers.
Study: Mouse_Radiation - All values are integers.
Study: Mouse_Post_Sendai - All values are integers.
Study: Mouse_TumorKO - All values are integers.
Study: Mouse_fibrosis - All values are integers.
Study: Mouse_asthma - All values are integers.
Study: Mouse_fibro_age - All values are integers.
Study: Mouse_influenza - All values are integers.


In [25]:
print(sets[0].shape)
covid.shape

(8265, 6000)


(8265, 33260)

In [ ]:
for i in range(0,len(sets)):
	temp = sets[i]
	sc.pp.normalize_total(temp, target_sum=1e4)
	sc.pp.log1p(temp)
	sc.pp.highly_variable_genes(temp, min_mean=0.0125, max_mean=3, min_disp=0.5,span=0.3,n_bins=20)
	sc.pp.highly_variable_genes(temp,n_top_genes=6000)
	#this plants the hvg under the var of the data
	temp = temp[:,temp.var.highly_variable]
	sets[i] = temp
	sets[i].var['highly_variable'] = temp.var.highly_variable
	vnames.append(set(sets[i].var_names))

for i in range(0,len(sets)):
    sets[i].X.data = sets[i].layers['raw'].data.copy()

check_non_integer_values(sets)

from itertools import combinations
for i in range(0,len(sets)-1):
	gene = vnames[i] & vnames[i+1]

comboSize = len(vnames)+1
while len(gene) < 2000:
	comboSize -= 1
	for combo in combinations(range(len(vnames)),comboSize):
		intersection = vnames[combo[0]]
		for i in combo[1:]: intersection = intersection & vnames[i]
		gene = gene.union(intersection)
		if len(gene) >= 2000:
			break
		#print(" and ".join(f"Set{i+1}" for i in combo),"=",len(intersection))

#inter = 2070
inter =list(gene)
if len(gene) > 2000:
	rem = len(gene) - 2000
	del inter[-rem:]

df = pd.DataFrame(inter, columns=['GeneSymbol'])

In [ ]:
Using the newly found HVG, we now add the HVG to each dataset

In [39]:
for i in range(0,len(sets)):
	genes = sets[i].var.index[sets[i].var.index.isin(inter)].tolist(){
	adata_sub = sets[i][:,genes].copy()
	genes_to_add = set(inter).difference(set(adata_sub.var_names))
	genes_to_add = list(genes_to_add)
	if genes_to_add != []:
		df_padding = pd.DataFrame(data=np.zeros((adata_sub.shape[0],len(genes_to_add))),index=adata_sub.obs_names, columns=genes_to_add)
		adata_padding = sc.AnnData(df_padding)
		adata_sub = anndata.concat([adata_sub, adata_padding], axis=1, index_unique=None, merge='unique')
	sets[i] = adata_sub[:,inter].copy()


for i in range(0,len(sets)):
	for col in sets[i].obs.columns:
		if pd.api.types.is_object_dtype(sets[i].obs[col]):
			sets[i].obs[col]=sets[i].obs[col].astype(str)
	#sets[i].var['features-1'] = sets[i].var['features-1'].astype(str)

for i in range(0,len(sets)):
    try:sets[i].__dict__['_raw'].__dict__['_var'] = sets[i].__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
    except:
        pass
        

In [ ]:
Saved for integration

In [40]:
for idx, adata_obj in enumerate(sets):
     print(f"Anndata object {idx + 1}:")
     print(f"Number of observations (cells): {adata_obj.n_obs}")
     print(f"Number of variables (genes/features): {adata_obj.n_vars}")
     print(f"Number of layers: {len(adata_obj.layers)}")
     print("\n")


Anndata object 1:
Number of observations (cells): 8265
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 2:
Number of observations (cells): 6567
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 3:
Number of observations (cells): 9295
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 4:
Number of observations (cells): 7159
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 5:
Number of observations (cells): 8233
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 6:
Number of observations (cells): 5376
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 7:
Number of observations (cells): 24214
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 8:
Number of observations (cells): 7413
Number of variables (genes/features): 2000
Number of layers: 0


Anndata object 9:
Number of observation

In [43]:
for i in range(0,len(sets)):
    study_value = sets[i].obs['study'].iloc[0]
    variable_name = str(study_value)
    sets[i].write(f"Data/{variable_name}_2k_h.h5ad")


Saving each set independantly as their healthy and disease variant for later integration